In [1]:
from dotenv import load_dotenv
from typing import Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel
from typing_extensions import TypedDict
import getpass
import os
import re

load_dotenv()

# Load API Key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)


# ---------------------------- CLASSIFIER MODEL ---------------------------- #

class MessageClassifier(BaseModel):
    message_type: Literal["emotional", "logical"]
    solution_count: int


class State(TypedDict):
    messages: Annotated[list, add_messages]
    message_type: str | None
    solution_count: int | None
    next: str | None


graph_builder = StateGraph(State)


# ------------------ EXTRACT NUMBER OF SOLUTIONS ------------------ #

def extract_solution_count(text: str) -> int:
    text = text.lower().strip()

    number_map = {
        "one": 1, "1": 1,
        "two": 2, "2": 2,
        "three": 3, "3": 3,
        "four": 4, "4": 4,
        "five": 5, "5": 5,
        "six": 6, "6": 6,
        "seven": 7, "7": 7,
        "eight": 8, "8": 8,
        "nine": 9, "9": 9,
        "ten": 10, "10": 10
    }

    for word, val in number_map.items():
        if word in text:
            return min(val, 10)

    m = re.search(r"give me (\d+)", text)
    if m:
        return min(int(m.group(1)), 10)

    if "many" in text:
        return 6
    if "some" in text:
        return 4
    if "couple" in text:
        return 2

    return 0


# ---------------------------- CLASSIFY NODE ---------------------------- #

def classify_message(state: State):
    last_message = state["messages"][-1]
    msg_text = last_message.content.lower()

    sol_count = extract_solution_count(msg_text)

    classifier_llm = llm.with_structured_output(MessageClassifier)

    result = classifier_llm.invoke([
        {
            "role": "system",
            "content": """
            Classify message as:
            - emotional: feelings, stress, sadness, overwhelm.
            - logical: when user directly asks for solutions.

            If user asks for ANY number of solutions,
            ALWAYS classify as logical.
            """
        },
        {"role": "user", "content": last_message.content}
    ])

    if sol_count > 0:
        result.message_type = "logical"

    return {
        "message_type": result.message_type,
        "solution_count": sol_count
    }


# ---------------------------- HISTORY BUILDER ---------------------------- #

def build_history(state: State):
    return [
        {"role": msg.type, "content": msg.content}
        for msg in state["messages"]
    ]


# ---------------------------- THERAPIST AGENT ---------------------------- #

def therapist_agent(state: State):
    history = build_history(state)

    reply = llm.invoke([
        {
            "role": "system",
            "content": """
            You are a compassionate therapist.
            - Empathize deeply.
            - Validate feelings.
            - NO step-by-step logical solutions.
            - Speak warmly, supportively.
            """
        },
        *history
    ])

    return {"messages": [{"role": "assistant", "content": reply.content}]}


# ---------------------------- LOGICAL AGENT ---------------------------- #

def logical_agent(state: State):
    history = build_history(state)
    sol_count = state.get("solution_count", 0)

    if sol_count <= 0:
        sol_count = 1  # default

    reply = llm.invoke([
        {
            "role": "system",
            "content": f"""
            You are a logical assistant.

            User requested EXACTLY {sol_count} solutions.
            - Give exactly {sol_count} solutions.
            - Each solution must be normal length (3–5 lines).
            - Number them clearly 1, 2, 3...
            - Be clear, practical, and helpful.
            """
        },
        *history
    ])

    return {"messages": [{"role": "assistant", "content": reply.content}]}


# ---------------------------- ROUTER NODE ---------------------------- #

def router(state: State):
    return {"next": state["message_type"]}


# ---------------------------- GRAPH BUILD ---------------------------- #

graph_builder.add_node("classify", classify_message)
graph_builder.add_node("therapist", therapist_agent)
graph_builder.add_node("logical", logical_agent)
graph_builder.add_node("router", router)

graph_builder.add_edge(START, "classify")
graph_builder.add_edge("classify", "router")

graph_builder.add_conditional_edges(
    "router",
    lambda state: state["next"],
    {
        "emotional": "therapist",
        "logical": "logical"
    }
)

graph_builder.add_edge("therapist", END)
graph_builder.add_edge("logical", END)

graph = graph_builder.compile()


# ---------------------------- FINAL CHAT LOOP ---------------------------- #

def run_chat():
    state: State = {
        "messages": [],
        "message_type": None,
        "solution_count": 0,
        "next": None
    }

    user_label = "𝗨𝘀𝗲𝗿"
    assistant_label = "𝗔𝘀𝘀𝗶𝘀𝘁𝗮𝗻𝘁"

    print("\n=== Adaptive AI Chat Started (type 'exit' to quit) ===\n")

    while True:
        user_input = input(f"{user_label}: ").strip()

        if user_input.lower() in ["exit", "quit"]:
            print("\nChat ended.")
            break

        state["messages"].append({"role": "user", "content": user_input})

        state = graph.invoke(state)

        assistant_message = state["messages"][-1].content
        print(f"{assistant_label}: {assistant_message}\n")


if __name__ == "__main__":
    run_chat()


Enter your Google AI API key:  ········



=== Adaptive AI Chat Started (type 'exit' to quit) ===



𝗨𝘀𝗲𝗿:  i am so stressful


𝗔𝘀𝘀𝗶𝘀𝘁𝗮𝗻𝘁: Oh, my dear, I hear you. "So stressful" – that sounds like a heavy weight you're carrying right now, and I'm so sorry you're feeling that way. It takes a lot to even say those words, and I want you to know I'm here to listen.

Please, just take a moment. It's completely understandable to feel overwhelmed when stress builds up. You don't have to explain anything right away, just know that your feelings are valid, and it's okay to feel exactly what you're feeling. I'm here with you.



𝗨𝘀𝗲𝗿:  because i have exam tomorrow and didnt start prepartion yet


...

𝗔𝘀𝘀𝗶𝘀𝘁𝗮𝗻𝘁: Here are the top two best solutions for your situation:

1.  **Prioritize High-Yield Content:** Immediately identify and focus on the most critical topics, definitions, formulas, or problem types that are most likely to appear on the exam. Use past papers, syllabus outlines, or chapter summaries to quickly pinpoint these "high-yield" areas, aiming for a basic understanding rather than deep mastery.

2.  **Implement a Micro-Study Plan & Strategic Rest:** Break down your remaining time into very short, focused study blocks (e.g., 25-30 minutes) with short breaks. Crucially, ensure you get at least 4-5 hours of sleep tonight. An exhausted brain performs poorly, and even a little rest will significantly improve your ability to recall information and think clearly during the exam.



𝗨𝘀𝗲𝗿:  thanks


𝗔𝘀𝘀𝗶𝘀𝘁𝗮𝗻𝘁: You are so very welcome, my dear. I truly hope these suggestions can offer a little bit of clarity and ease some of that immense pressure you're feeling.

Remember, even in the most challenging moments, taking a deep breath and focusing on one small step at a time can make a difference. You're facing a tough situation with courage, and I'm sending you all my support.

Please, be kind to yourself through this. You've got this.



𝗨𝘀𝗲𝗿:  exit



Chat ended.


In [5]:
import os
os.getcwd()


'C:\\Users\\veera'